<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [131]:
import sqlalchemy
from sqlalchemy import create_engine

In [61]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) #open a connection to the database
c = conn.cursor() #creates a cursor object. this step always needed to interact with the db

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [62]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
#conn.commit() 

In [63]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);') ## this is the table schema, features/variables and data types after
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [64]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT into houses values (?, ?, ?, ?, ?)', last_sale)
# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [65]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
c.executemany('INSERT into houses values (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Select all rows from houses

In [66]:
# A
for row in c.execute('SELECT * FROM houses ORDER BY price'):
    print(row)

(4, 1505, 3, 90, 269000)
(3, 1870, 3, 14, 289000)
(2, 2390, 4, 34, 319000)
(1, 4000, 5, 22, 619000)





#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [67]:
import numpy
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('/Users/lawrenceyim/Documents/Institute of Data/Labs/sql/data/housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

In [68]:
help(genfromtxt)

Help on function genfromtxt in module numpy:

genfromtxt(fname, dtype=<class 'float'>, comments='#', delimiter=None, skip_header=0, skip_footer=0, converters=None, missing_values=None, filling_values=None, usecols=None, names=None, excludelist=None, deletechars=" !#$%&'()*+,-./:;<=>?@[\\]^{|}~", replace_space='_', autostrip=False, case_sensitive=True, defaultfmt='f%i', unpack=None, usemask=False, loose=True, invalid_raise=True, max_rows=None, encoding='bytes', *, like=None)
    Load data from a text file, with missing values handled as specified.
    
    Each line past the first `skip_header` lines is split at the `delimiter`
    character, and characters following the `comments` character are discarded.
    
    Parameters
    ----------
    fname : file, str, pathlib.Path, list of str, generator
        File, filename, list, or generator to read.  If the filename
        extension is `.gz` or `.bz2`, the file is first decompressed. Note
        that generators must return byte strin

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [69]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [70]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [71]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [72]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [73]:
#The SQLite DELETE statement allows you to delete one row, multiple rows, and all rows in a table. 
#where would specify the condition
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [22]:
#checking to see how many bedrooms there are in dataset
for row in c.execute('SELECT * FROM houses'):
    print(row)


(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)
(5, 2104, 3, 70, 399900)
(6, 1600, 3, 28, 329900)
(7, 2400, 3, 44, 369000)
(8, 1416, 2, 49, 232000)
(9, 3000, 4, 75, 539900)
(10, 1985, 4, 61, 299900)
(11, 1534, 3, 12, 314900)
(12, 1427, 3, 57, 198999)
(13, 1380, 3, 14, 212000)
(14, 1494, 3, 15, 242500)
(15, 1940, 4, 7, 239999)
(16, 2000, 3, 27, 347000)
(17, 1890, 3, 45, 329999)
(18, 4478, 5, 49, 699900)
(19, 1268, 3, 58, 259900)
(20, 2300, 4, 77, 449900)
(21, 1320, 2, 62, 299900)
(22, 1236, 3, 78, 199900)
(23, 2609, 4, 5, 499998)
(24, 3031, 4, 21, 599000)
(25, 1767, 3, 44, 252900)
(26, 1888, 2, 79, 255000)
(27, 1604, 3, 13, 242900)
(28, 1962, 4, 53, 259900)
(29, 3890, 3, 36, 573900)
(30, 1100, 3, 60, 249900)
(31, 1458, 3, 29, 464500)
(32, 2526, 3, 13, 469000)
(33, 2200, 3, 28, 475000)
(34, 2637, 3, 25, 299900)
(35, 1839, 2, 40, 349900)
(36, 1000, 1, 5, 169900)
(37, 2040, 4, 75, 314900)
(38, 3137, 3, 67, 579900)
(39, 1811, 4, 24, 285900

In [74]:
# USE A WHERE CLAUSE TO FIND CONDITION
c.execute("SELECT * FROM houses WHERE bdrms = '4'")
c.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [75]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [76]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [77]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('/Users/lawrenceyim/Documents/Institute of Data/Labs/sql/data/housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [78]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [79]:
results = c.execute("SELECT AVG(sqft) AS avg_sqft FROM houses_pandas")
results.fetchall()

[(2000.6808510638298,)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [80]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [120]:
# Reading CSV to Dataframe P12-ListOfOrders
df_orders = pd.read_csv('/Users/lawrenceyim/Documents/Institute of Data/Labs/sql/data/P12-ListOfOrders.csv', encoding = 'utf-8')
df_orders_break_down =  pd.read_csv('/Users/lawrenceyim/Documents/Institute of Data/Labs/sql/data/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [121]:
df_orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [122]:
df_orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [123]:
# ANSWER
df_orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [124]:
# ANSWER
df_orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [125]:
# ANSWER
df_orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [138]:
# ANSWER
df_orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite


In [150]:
#create connection to db with sqlite3
db_connection = sqlite3.connect('sqlite_db2')
#use df.to_sql https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
#syntax is dfname.tosql('table name', sqlite connection)
#df_orders.to_sql('orders', con=db_connection)
#df_orders_break_down.to_sql('ordersbreakdown', con=db_connection)

In [143]:
conn.commit()

In [117]:
#use pandas readsql to check and read db, syntax is pd.read_sql('tablename', engine)


In [151]:
# Establishing Local DB connection
#sqlite_db2 = 'sqlite_db2.sqlite' #creates the db 'sqlite_db2'
db_connection = sqlite3.connect('sqlite_db2')
c2 = db_connection.cursor()
# Save these two dataframes as a table in sqlite
#create table orders first
#c2.execute('CREATE TABLE orders (field1 INTEGER PRIMARY KEY, order_id TEXT, date TEXT, customer_name TEXT, country TEXT);') ## this is the table schema, features/variables and data types after

#create 2nd table
#c2.execute('CREATE TABLE orderbreakdown (field1 INTEGER PRIMARY KEY, order_id TEXT, discount REAL, sales REAL, quantity INTEGER, category TEXT);') ## this is the table schema, features/variables and data types after
#conn.commit()
# import into nparray of ints, then convert to list of lists:
#data_orders = genfromtxt('/Users/lawrenceyim/Documents/Institute of Data/Labs/sql/data/P12-ListOfOrders.csv', dtype = 'i8', delimiter = ',', skip_header = 1).tolist()


#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [153]:
# Select first 10 rows of all columns in orders table
query = """
        SELECT * 
        FROM orders 
        LIMIT 10;
        """

c2.execute(query)
c2.fetchall()

[(0, 'IT-2011-3647632', '2011-01-01', 'Eugene Moren', 'Sweden'),
 (1, 'ES-2011-4869686', '2011-01-03', 'Dorothy Dickinson', 'United Kingdom'),
 (2, 'ES-2011-4939443', '2011-01-04', 'Arthur Prichep', 'France'),
 (3, 'IT-2011-2942451', '2011-01-04', 'Grant Thornton', 'United Kingdom'),
 (4, 'ES-2011-3848439', '2011-01-05', 'Michael Granlund', 'France'),
 (5, 'ES-2011-5433855', '2011-01-07', 'Dave Poirier', 'France'),
 (6, 'IT-2011-4546695', '2011-01-08', 'Darren Powers', 'France'),
 (7, 'ES-2011-1138719', '2011-01-11', 'Eric Murdock', 'Italy'),
 (8, 'ES-2011-1466305', '2011-01-11', 'Mick Brown', 'Austria'),
 (9, 'ES-2011-4359424', '2011-01-11', 'Dorothy Dickinson', 'Spain')]

In [157]:
# Select Number of Orders for Each Customer
query2 = """
        SELECT COUNT(customer_name), customer_name 
        FROM orders
        GROUP BY customer_name
        ORDER BY customer_name;
        """
c2.execute(query2)
c2.fetchall()

[(11, 'Aaron Bergman'),
 (7, 'Aaron Hawkins'),
 (12, 'Aaron Smayling'),
 (4, 'Adam Bellavance'),
 (8, 'Adam Hart'),
 (5, 'Adam Shillingsburg'),
 (5, 'Adrian Barton'),
 (5, 'Adrian Hane'),
 (2, 'Adrian Shami'),
 (6, 'Aimee Bixby'),
 (9, 'Alan Barnes'),
 (5, 'Alan Dominguez'),
 (7, 'Alan Haines'),
 (9, 'Alan Hwang'),
 (6, 'Alan Schoenberger'),
 (5, 'Alan Shonely'),
 (6, 'Alejandro Ballentine'),
 (2, 'Alejandro Grove'),
 (7, 'Alejandro Savely'),
 (7, 'Aleksandra Gannaway'),
 (5, 'Alex Avila'),
 (8, 'Alex Grayson'),
 (7, 'Alex Russell'),
 (7, 'Alice McCarthy'),
 (2, 'Allen Armold'),
 (2, 'Allen Goldenen'),
 (6, 'Allen Rosenblatt'),
 (5, 'Alyssa Crouse'),
 (3, 'Alyssa Tate'),
 (1, 'Amy Cox'),
 (2, 'Amy Hunt'),
 (6, 'Andrew Allen'),
 (10, 'Andrew Gjertsen'),
 (5, 'Andrew Roberts'),
 (8, 'Andy Gerbode'),
 (1, 'Andy Reiter'),
 (2, 'Andy Yotov'),
 (4, 'Anemone Ratner'),
 (5, 'Angele Hood'),
 (5, 'Ann Blume'),
 (4, 'Ann Chong'),
 (4, 'Ann Steele'),
 (7, 'Anna Andreadi'),
 (5, 'Anna Chung'),
 (6,

#### 6. Select Number of Customers for Each Country

In [158]:
# Select Number of Customers for Each Country
query3 = """
        SELECT COUNT(country), country
        FROM orders
        GROUP BY country
        ORDER BY country;
        """
c2.execute(query3)
c2.fetchall()

[(135, 'Austria'),
 (68, 'Belgium'),
 (29, 'Denmark'),
 (34, 'Finland'),
 (991, 'France'),
 (806, 'Germany'),
 (50, 'Ireland'),
 (493, 'Italy'),
 (194, 'Netherlands'),
 (37, 'Norway'),
 (37, 'Portugal'),
 (403, 'Spain'),
 (100, 'Sweden'),
 (40, 'Switzerland'),
 (700, 'United Kingdom')]

#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [161]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
query4 = """
        SELECT discount, quantity 
        FROM ordersbreakdown;
        """
c2.execute(query4)
c2.fetchall()

[(0.5, 3),
 (0.0, 7),
 (0.0, 3),
 (0.5, 2),
 (0.5, 2),
 (0.0, 3),
 (0.0, 4),
 (0.1, 1),
 (0.0, 3),
 (0.1, 4),
 (0.6, 6),
 (0.5, 5),
 (0.0, 3),
 (0.0, 2),
 (0.6, 2),
 (0.0, 1),
 (0.0, 2),
 (0.0, 5),
 (0.0, 3),
 (0.5, 2),
 (0.0, 5),
 (0.1, 5),
 (0.0, 5),
 (0.0, 2),
 (0.0, 2),
 (0.5, 3),
 (0.5, 4),
 (0.5, 2),
 (0.5, 2),
 (0.5, 5),
 (0.0, 11),
 (0.0, 3),
 (0.0, 6),
 (0.1, 1),
 (0.0, 2),
 (0.0, 5),
 (0.0, 2),
 (0.0, 2),
 (0.5, 3),
 (0.0, 8),
 (0.5, 1),
 (0.0, 3),
 (0.4, 3),
 (0.0, 3),
 (0.0, 2),
 (0.0, 2),
 (0.0, 2),
 (0.0, 2),
 (0.0, 5),
 (0.0, 1),
 (0.4, 7),
 (0.6, 1),
 (0.6, 3),
 (0.0, 4),
 (0.0, 5),
 (0.0, 3),
 (0.0, 1),
 (0.0, 7),
 (0.0, 1),
 (0.0, 10),
 (0.0, 1),
 (0.0, 4),
 (0.0, 3),
 (0.0, 3),
 (0.5, 3),
 (0.1, 1),
 (0.0, 3),
 (0.0, 3),
 (0.0, 2),
 (0.1, 2),
 (0.0, 1),
 (0.0, 2),
 (0.1, 2),
 (0.5, 1),
 (0.5, 1),
 (0.5, 8),
 (0.5, 5),
 (0.0, 1),
 (0.5, 2),
 (0.65, 6),
 (0.65, 3),
 (0.1, 1),
 (0.0, 2),
 (0.2, 4),
 (0.0, 1),
 (0.0, 4),
 (0.5, 6),
 (0.1, 3),
 (0.0, 4),
 (0.0, 4),
 (0.15

#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [162]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
query5 = """
        SELECT discount, sales, quantity, sales*quantity-discount as total_price
        FROM ordersbreakdown;
        """
c2.execute(query5)
c2.fetchall()

[(0.5, 45.0, 3, 134.5),
 (0.0, 854.0, 7, 5978.0),
 (0.0, 140.0, 3, 420.0),
 (0.5, 27.0, 2, 53.5),
 (0.5, 17.0, 2, 33.5),
 (0.0, 90.0, 3, 270.0),
 (0.0, 207.0, 4, 828.0),
 (0.1, 155.0, 1, 154.9),
 (0.0, 33.0, 3, 99.0),
 (0.1, 716.0, 4, 2863.9),
 (0.6, 987.0, 6, 5921.4),
 (0.5, 116.0, 5, 579.5),
 (0.0, 1384.0, 3, 4152.0),
 (0.0, 103.0, 2, 206.0),
 (0.6, 268.0, 2, 535.4),
 (0.0, 55.0, 1, 55.0),
 (0.0, 97.0, 2, 194.0),
 (0.0, 40.0, 5, 200.0),
 (0.0, 22.0, 3, 66.0),
 (0.5, 34.0, 2, 67.5),
 (0.0, 290.0, 5, 1450.0),
 (0.1, 552.0, 5, 2759.9),
 (0.0, 257.0, 5, 1285.0),
 (0.0, 17.0, 2, 34.0),
 (0.0, 522.0, 2, 1044.0),
 (0.5, 9.0, 3, 26.5),
 (0.5, 22.0, 4, 87.5),
 (0.5, 50.0, 2, 99.5),
 (0.5, 198.0, 2, 395.5),
 (0.5, 31.0, 5, 154.5),
 (0.0, 224.0, 11, 2464.0),
 (0.0, 486.0, 3, 1458.0),
 (0.0, 440.0, 6, 2640.0),
 (0.1, 249.0, 1, 248.9),
 (0.0, 21.0, 2, 42.0),
 (0.0, 170.0, 5, 850.0),
 (0.0, 55.0, 2, 110.0),
 (0.0, 527.0, 2, 1054.0),
 (0.5, 17.0, 3, 50.5),
 (0.0, 108.0, 8, 864.0),
 (0.5, 26.0, 1, 2

#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [165]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
query6 = """
        SELECT discount, sales, quantity, sales*quantity-discount as total_price
        FROM ordersbreakdown
        WHERE total_price > 100
        ORDER BY total_price;
        """
c2.execute(query6)
c2.fetchall()

[(0.5, 51.0, 2, 101.5),
 (0.5, 51.0, 2, 101.5),
 (0.5, 34.0, 3, 101.5),
 (0.5, 51.0, 2, 101.5),
 (0.5, 51.0, 2, 101.5),
 (0.5, 51.0, 2, 101.5),
 (0.5, 102.0, 1, 101.5),
 (0.4, 34.0, 3, 101.6),
 (0.4, 34.0, 3, 101.6),
 (0.1, 34.0, 3, 101.9),
 (0.1, 34.0, 3, 101.9),
 (0.1, 34.0, 3, 101.9),
 (0.0, 34.0, 3, 102.0),
 (0.0, 51.0, 2, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 51.0, 2, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 51.0, 2, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 51.0, 2, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 34.0, 3, 102.0),
 (0.0, 51.0, 2,

#### 8. Select All Customer And The Product They Have Bought

In [166]:
# Select All Customer And The Product They Have Bought
query7 = """
        SELECT *
        FROM orders a
        INNER JOIN ordersbreakdown b
        ON a.order_id = b.order_id 
        GROUP BY a.customer_name;
        """
c2.execute(query7)
c2.fetchall()

[(99,
  'ES-2011-3528133',
  '2011-03-21',
  'Aaron Bergman',
  'Italy',
  190,
  'ES-2011-3528133',
  'Harbour Creations Round Labels, Laser Printer Compatible',
  0.0,
  47.0,
  7,
  'Office Supplies'),
 (268,
  'IT-2011-4320455',
  '2011-06-17',
  'Aaron Hawkins',
  'France',
  525,
  'IT-2011-4320455',
  'Xerox Note Cards, Recycled',
  0.0,
  210.0,
  8,
  'Office Supplies'),
 (100,
  'ES-2011-5083176',
  '2011-03-21',
  'Aaron Smayling',
  'Switzerland',
  193,
  'ES-2011-5083176',
  'Binney & Smith Pens, Fluorescent',
  0.0,
  26.0,
  2,
  'Office Supplies'),
 (1652,
  'ES-2012-1205030',
  '2012-12-19',
  'Adam Bellavance',
  'United Kingdom',
  3242,
  'ES-2012-1205030',
  'Sauder Classic Bookcase, Mobile',
  0.1,
  1579.0,
  4,
  'Furniture'),
 (248,
  'IT-2011-4648014',
  '2011-06-10',
  'Adam Hart',
  'Italy',
  489,
  'IT-2011-4648014',
  'Cardinal Binder Covers, Economy',
  0.0,
  48.0,
  4,
  'Office Supplies'),
 (198,
  'IT-2011-1521406',
  '2011-05-21',
  'Adam Shillings

#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [181]:
# Select Number of 'Furniture' Orders For Each Country
query7 = """
        SELECT COUNT(a.country), a.country
        FROM orders a
        INNER JOIN ordersbreakdown b
        ON a.order_id = b.order_id
        WHERE b.category LIKE 'furniture'
        GROUP BY a.country
        ORDER BY COUNT(a.country) DESC;
        """
c2.execute(query7)
c2.fetchall()

[(299, 'France'),
 (264, 'Germany'),
 (180, 'United Kingdom'),
 (136, 'Italy'),
 (123, 'Spain'),
 (76, 'Netherlands'),
 (40, 'Austria'),
 (36, 'Sweden'),
 (20, 'Belgium'),
 (16, 'Finland'),
 (12, 'Ireland'),
 (11, 'Switzerland'),
 (10, 'Portugal'),
 (9, 'Denmark'),
 (6, 'Norway')]

#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [189]:
# Select Number of 'Furniture' Orders For The Country Denmark
query8 = """
        SELECT a.country, COUNT(a.country)
        FROM orders a
        INNER JOIN ordersbreakdown b
        ON a.order_id = b.order_id
        WHERE a.country LIKE 'Denmark'
        AND b.category LIKE 'Furniture';
        """
c2.execute(query8)
c2.fetchall()

[('Denmark', 9)]

#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [195]:
# Select Total Sales With Discount and Without Discount for Each Country
query9 = """
        SELECT a.country, 
        SUM(b.sales*b.quantity) as total_sales,
        SUM(b.sales*b.quantity-b.discount) as total_sales_discount
        FROM orders a
        INNER JOIN ordersbreakdown b
        GROUP BY a.country;
        """
c2.execute(query9)
c2.fetchall()

[('Austria', 1606362705.0, 1606243155.752188),
 ('Belgium', 809130844.0, 809070626.5990552),
 ('Denmark', 345070507.0, 345044826.04996186),
 ('Finland', 404565422.0, 404535313.2998587),
 ('France', 11791892153.0, 11791014572.746815),
 ('Germany', 9590580298.0, 9589866544.571127),
 ('Ireland', 594949150.0, 594904872.4995071),
 ('Italy', 5866198619.0, 5865762042.830478),
 ('Netherlands', 2308402702.0, 2308230905.3072987),
 ('Norway', 440262371.0, 440229605.6497968),
 ('Portugal', 440262371.0, 440229605.6497968),
 ('Spain', 4795290149.0, 4794933272.360192),
 ('Sweden', 1189898300.0, 1189809744.999299),
 ('Switzerland', 475959320.0, 475923897.9997349),
 ('United Kingdom', 8329288100.0, 8328668214.912134)]

#### 11.A Select Total Quantity, Total Sales for Each Country

In [196]:
# Select Total Quantity, Total Sales for Each Country
query10 = """
        SELECT a.country, 
        SUM(b.sales),
        SUM(b.quantity)
        FROM orders a
        INNER JOIN ordersbreakdown b
        GROUP BY a.country;
        """
c2.execute(query10)
c2.fetchall()

[('Austria', 317045070.0, 4097790),
 ('Belgium', 159696776.0, 2064072),
 ('Denmark', 68105978.0, 880266),
 ('Finland', 79848388.0, 1032036),
 ('France', 2327345662.0, 30080814),
 ('Germany', 1892876492.0, 24465324),
 ('Ireland', 117424100.0, 1517700),
 ('Italy', 1157801626.0, 14964522),
 ('Netherlands', 455605508.0, 5888676),
 ('Norway', 86893834.0, 1123098),
 ('Portugal', 86893834.0, 1123098),
 ('Spain', 946438246.0, 12232662),
 ('Sweden', 234848200.0, 3035400),
 ('Switzerland', 93939280.0, 1214160),
 ('United Kingdom', 1643937400.0, 21247800)]

#### 11.B Select Top 3 Country Based on Sales

In [199]:
# Select Top 3 Country Based on Sales
query11 = """
        SELECT a.country, 
        SUM(b.sales)
        FROM orders a
        INNER JOIN ordersbreakdown b
        GROUP BY a.country
        ORDER BY a.country DESC
        LIMIT 3;
        """
c2.execute(query11)
c2.fetchall()

[('United Kingdom', 1643937400.0),
 ('Switzerland', 93939280.0),
 ('Sweden', 234848200.0)]

#### 11.C Select Bottom 3 Country Based On Quantities

In [200]:
# Select Bottom 3 Country Based On Quantities
query12 = """
        SELECT a.country, 
        SUM(b.quantity)
        FROM orders a
        INNER JOIN ordersbreakdown b
        GROUP BY a.country
        ORDER BY a.country DESC
        LIMIT 3;
        """
c2.execute(query12)
c2.fetchall()

[('United Kingdom', 21247800), ('Switzerland', 1214160), ('Sweden', 3035400)]

#### 12. Select Average Sales By Categroy For The Country 'France'

In [201]:
# Select Average Sales By Categroy For The Country 'France'
query13 = """
        SELECT a.country, 
        AVG(b.sales)
        FROM orders a
        INNER JOIN ordersbreakdown b
        WHERE a.country LIKE 'France';
        """
c2.execute(query13)
c2.fetchall()

[('France', 291.84565676649686)]

#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [204]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
query14 = """
        SELECT a.country, 
        SUM(b.sales) as total_sales
        FROM orders a
        INNER JOIN ordersbreakdown b
        GROUP BY a.country
        ORDER BY AVG(b.sales) DESC
        LIMIT 1;
        """
c2.execute(query14)
c2.fetchall()

[('United Kingdom', 1643937400.0)]

In [205]:
conn.commit()

### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



